Keras w/TF - 48s

Keras w/CNTK - 75s

Tensorflow - 22s

MXNet - 46s

PyTorch - 44s

CNTK - 27s

Caffe2 - 48s

In [ ]:
import os
import sys
import numpy as np
# Kill all GPUs ...
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
BATCH_SIZE = 128
RESNET_FEATURES = 2048
BATCHES = 2

In [ ]:
def give_fake_data():
    """ Create an array of fake data to run inference on"""
    np.random.seed(0)
    dta = np.random.rand(BATCH_SIZE*BATCHES, 224, 224, 3).astype(np.float32)
    return dta, np.swapaxes(dta, 1, 3)

In [ ]:
# Create batches of fake data
fake_input_data_cl, fake_input_data_cf = give_fake_data()
print(fake_input_data_cl.shape, fake_input_data_cf.shape)

In [ ]:
def yield_mb(X, batchsize):
    """ Function yield mini_batches of data"""
    for i in range(len(X)//batchsize):
        yield i, X[i*batchsize:(i+1)*batchsize]

## 1. Keras defaults

In [ ]:
os.environ['KERAS_BACKEND'] = "tensorflow"
import keras as K
import tensorflow as tf
from keras.applications.resnet50 import ResNet50

In [ ]:
print("Keras: ", K.__version__)
print("Tensorflow: ", tf.__version__)
print("Keras using {}".format(K.backend.backend()))
print("Keras channel ordering is {}".format(K.backend.image_data_format()))

In [ ]:
# Set channels last (default)
K.backend.set_image_data_format('channels_last')

In [ ]:
def predict_fn(classifier, data, batchsize):
    """ Return features from classifier """
    out = np.zeros((len(data), RESNET_FEATURES), np.float32)
    for idx, dta in yield_mb(data, batchsize):
        out[idx*batchsize:(idx+1)*batchsize] = classifier.predict_on_batch(dta).squeeze()
    return out

In [ ]:
# Load ResNet50 bottom from Keras
model = ResNet50(include_top=False)
model.summary()

In [ ]:
%%time
# 48.7s
# Forward-pass batches
features = predict_fn(model, fake_input_data_cl, BATCH_SIZE)

## 2. Keras with CNTK

In [ ]:
os.environ['KERAS_BACKEND'] = "cntk"
import keras as K
import cntk as C

In [ ]:
print("Keras: ", K.__version__)
print("CNTK: ", C.__version__)
print("Keras using {}".format(K.backend.backend()))
# Set channels first (default for CNTK)
K.backend.set_image_data_format('channels_first')
print("Keras channel ordering is {}".format(K.backend.image_data_format()))
from keras.applications.resnet50 import ResNet50

In [ ]:
def predict_fn(classifier, data, batchsize):
    """ Return features from classifier """
    out = np.zeros((len(data), RESNET_FEATURES), np.float32)
    for idx, dta in yield_mb(data, batchsize):
        out[idx*batchsize:(idx+1)*batchsize] = classifier.predict_on_batch(dta).squeeze()
    return out

In [ ]:
# Load ResNet50 bottom from Keras
model = ResNet50(include_top=False, input_shape=(3,224,224))
#model.summary()

In [ ]:
%%time
# 1min15s
features = predict_fn(model, fake_input_data_cf, BATCH_SIZE)
print("No effect really when on CPU ...")

## 3. Tensorflow

In [ ]:
# Lots of models! (Including ResNet V2)
# https://github.com/tensorflow/models/tree/master/research/slim

In [ ]:
%%bash
CHECKPOINT_DIR=/data/checkpoints
mkdir ${CHECKPOINT_DIR}
wget http://download.tensorflow.org/models/resnet_v1_50_2016_08_28.tar.gz
tar -xvf resnet_v1_50_2016_08_28.tar.gz
mv resnet_v1_50_2016_08_28.tar.gz ${CHECKPOINT_DIR}

In [ ]:
import tensorflow as tf
import tensorflow.contrib.slim
from tensorflow.contrib.slim.nets import resnet_v1

In [ ]:
# Placeholders
checkpoint_file = 'resnet_v1_50.ckpt'
input_tensor = tf.placeholder(tf.float32, shape=(None,224,224,3), name='input_image')

# Load the model
sess = tf.Session()
arg_scope = resnet_v1.resnet_arg_scope()
with tensorflow.contrib.slim.arg_scope(arg_scope):
    # Docstring ->
    #     num_classes: Number of predicted classes for classification tasks. If None
    #  we return the features before the logit layer.
    logits, end_points = resnet_v1.resnet_v1_50(input_tensor, is_training=False)
    
saver = tf.train.Saver()
saver.restore(sess, checkpoint_file)

In [ ]:
def predict_fn(classifier, data, batchsize):
    """ Return features from classifier """
    out = np.zeros((len(data), RESNET_FEATURES), np.float32)
    for idx, dta in yield_mb(data, batchsize):
        pred = sess.run(classifier, feed_dict={input_tensor: dta}).squeeze()
        out[idx*batchsize:(idx+1)*batchsize] = pred
    return out

In [ ]:
%%time
# 22.1s
features = predict_fn(logits, fake_input_data_cl, BATCH_SIZE)

## 4. MXNet

In [ ]:
import mxnet as mx
from collections import namedtuple
print("MXNet: ", mx.__version__)
Batch = namedtuple('Batch', ['data'])

In [ ]:
# Download Resnet weights
path='http://data.mxnet.io/models/imagenet/'
[mx.test_utils.download(path+'resnet/50-layers/resnet-50-symbol.json'),
 mx.test_utils.download(path+'resnet/50-layers/resnet-50-0000.params')]

In [ ]:
# Load model
sym, arg_params, aux_params = mx.model.load_checkpoint('resnet-50', 0)
# List the last 10 layers
all_layers = sym.get_internals()

In [ ]:
print(all_layers.list_outputs()[-10:])

In [ ]:
# Get last layer
fe_sym = all_layers['flatten0_output']
# Initialise
fe_mod = mx.mod.Module(symbol=fe_sym, context=mx.cpu(), label_names=None)
fe_mod.bind(for_training=False, data_shapes=[('data', (BATCH_SIZE,3,224,224))])
fe_mod.set_params(arg_params, aux_params)

In [ ]:
def predict_fn(classifier, data, batchsize):
    """ Return features from classifier """
    out = np.zeros((len(data), RESNET_FEATURES), np.float32)
    for idx, dta in yield_mb(data, batchsize):
        classifier.forward(Batch(data=[mx.nd.array(dta)]))
        out[idx*batchsize:(idx+1)*batchsize] = classifier.get_outputs()[0].asnumpy().squeeze()
    return out

In [ ]:
%%time
# 46.7s
# Forward-pass batches
features = predict_fn(fe_mod, fake_input_data_cf, BATCH_SIZE)

## 5. PyTorch

In [ ]:
import torch
from torch.autograd import Variable
import torchvision.models as models

In [ ]:
resnet50 = models.resnet50(pretrained=True)

In [ ]:
chopped_resnet50 = torch.nn.Sequential(*list(resnet50.children())[:-1])

In [ ]:
def predict_fn(classifier, data, batchsize):
    """ Return features from classifier """
    classifier.eval()
    out = np.zeros((len(data), RESNET_FEATURES), np.float32)
    for idx, dta in yield_mb(data, batchsize):
        pred = classifier(Variable(torch.FloatTensor(dta)))
        out[idx*batchsize:(idx+1)*batchsize] = pred.data.numpy().squeeze()
    return out

In [ ]:
%%time
# 44.1s
# Forward-pass batches
features = predict_fn(chopped_resnet50, fake_input_data_cf, BATCH_SIZE)

## 6. CNTK

In [ ]:
import cntk as C
from cntk import load_model, combine

In [ ]:
#%%bash
#wget https://www.cntk.ai/Models/CNTK_Pretrained/ResNet50_ImageNet_CNTK.model

In [ ]:
# Features (penultimate layer)
node_name = "z.x"
model_file = "ResNet50_ImageNet_CNTK.model"
# Load model
loaded_model  = load_model(model_file)
node_in_graph = loaded_model.find_by_name(node_name)
output_nodes  = combine([node_in_graph.owner])

In [ ]:
def predict_fn(classifier, data, batchsize):
    """ Return features from classifier """
    out = np.zeros((len(data), RESNET_FEATURES), np.float32)
    for idx, dta in yield_mb(data, batchsize):
        pred = classifier.eval(dta)
        out[idx*batchsize:(idx+1)*batchsize] = pred[0].squeeze()
    return out

In [ ]:
%%time
# 27.6s
# Forward-pass batches
features = predict_fn(output_nodes, fake_input_data_cf, BATCH_SIZE)

## 7. Caffe2

In [ ]:
import caffe2
from caffe2.proto import caffe2_pb2
from caffe2.python import core, workspace, models

In [ ]:
#%%bash
#wget https://github.com/leonardvandriel/caffe2_models/raw/master/model/resnet50_init_net.pb
#wget https://github.com/leonardvandriel/caffe2_models/raw/master/model/resnet50_predict_net.pb

In [ ]:
# Load model
with open("resnet50_init_net.pb", "rb") as f:
     init_net = f.read()
with open("resnet50_predict_net.pb", "rb") as f:
    predict_net = f.read()   

In [ ]:
workspace.RunNetOnce(init_net)
workspace.CreateNet(predict_net)
p = workspace.Predictor(init_net, predict_net)

In [ ]:
def predict_fn(classifier, data, batchsize):
    """ Return features from classifier """
    out = np.zeros((len(data), RESNET_FEATURES), np.float32)
    for idx, dta in yield_mb(data, batchsize):
        results = classifier.run([dta])
        # Last feature layer should be this average-pooling
        # I think ... "pool5"
        # Still running softmax so seems wasteful
        out[idx*batchsize:(idx+1)*batchsize] = workspace.FetchBlob('pool5').squeeze()
    return out

In [ ]:
%%time
# 48.5s ???? Something must be wrong
# Forward-pass batches
features = predict_fn(p, fake_input_data_cf, BATCH_SIZE)